In [2]:
import pandas as pd
from scholarly import scholarly
from habanero import Crossref
import os
import time

# Get article information from Google Scholar (author, year, title) and then use Crossref to get the DOI with those informations

In [3]:
def get_doi_from_query(query, email="votre_adresse@email.com"):
    """
    Recherche la première publication correspondant à 'query' sur Google Scholar,
    puis tente d'obtenir son DOI via l'API Crossref.
    
    Paramètres:
    -----------
    - query : str
        La requête à envoyer à Google Scholar (ex: titre, auteurs, mots-clés...).
    - email : str
        Adresse e-mail de contact pour l'API Crossref (optionnel).
    
    Retourne:
    ---------
    - Le DOI (sous forme de chaîne) s'il est trouvé, sinon None.
    """
    # Étape 1 : Recherche sur Google Scholar
    search_results = scholarly.search_pubs(query, patents=False)

    try:
        # On récupère le premier résultat
        first_result = next(search_results)
    except StopIteration:
        print("Aucun résultat trouvé sur Google Scholar.")
        return None

    # Extraction des champs utiles (auteur, année, titre)
    authors = first_result["bib"].get("author", [])
    pub_year = first_result["bib"].get("pub_year", "")
    title = first_result["bib"].get("title", "")
    
    # Étape 2 : Interrogation de Crossref
    cr = Crossref(mailto=email)
    
    # Construction d'une requête basée sur auteur(s), année et titre
    # On joint les auteurs avec une virgule et on ajoute le titre entre guillemets
    crossref_query = f"{', '.join(authors)} {pub_year} \"{title}\""
    
    try:
        # On limite à 1 résultat et on demande uniquement le champ DOI
        response = cr.works(query=crossref_query, limit=1, select="DOI")
        doi = response["message"]["items"][0]["DOI"]
        # Préfixe optionnel si vous souhaitez renvoyer l'URL complète du DOI
        doi_url = "https://doi.org/" + doi
        print(f"DOI found for {query}: {doi_url}")
        return doi_url
    except Exception as e:
        print("Impossible de récupérer le DOI sur Crossref :", e)
        return None



# Get DOI and save to references_with_doi.csv

In [ ]:
# Import the sheet
df = pd.read_csv("../3_ocr_results_clean_to_references/references_without_doi.csv")

# Add a column with the DOI
df['DOI'] = ""

# Create the references_with_doi.csv file if it doesn't exist
if not os.path.exists("references_with_doi.csv"):
    df.to_csv("references_with_doi.csv", index=False)
    print("File created")
else:
    print("File already exists")

# Complet the sheet_with_doi.csv file with the DOI
for index, row in df.iterrows():    
    if pd.isna(row['DOI']):
        print(f"Searching DOI for {row['Reference']}")
        df.at[index, 'DOI'] = get_doi_from_query(row['Reference'])
        df.to_csv("references_with_doi.csv", index=False)
        # Wait 10 seconds
        time.sleep(10)
    else:
        print(f"DOI already found for {row['Reference']}")